# Part 3: Single-View Geometry

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [13]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
from sympy.solvers import solve
from sympy import *
from numpy.linalg import norm


from PIL import Image

# Provided functions

In [14]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
       
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [15]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [16]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [17]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR IMPLEMENTATION>
    line1 = lines[:, 0]
    line2 = lines[:, 1]
    line3 = lines[:, 2]

    v1 = np.cross(line1, line2)
    v2 = np.cross(line2, line3)
    v3 = np.cross(line1, line3)

    v = (v1 + v2 + v3) / 3

    print(v[0] / v[2], v[1] / v[2])

    return v

In [18]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    # <YOUR IMPLEMENTATION>

    v1 = vpts[:, 0]
    v2 = vpts[:, 1]

    h = np.cross(v1, v2)
    h = h / np.sqrt(h[0]**2 + h[1]**2)

    print('horizon:')
    print(h)
    
    return h

In [19]:
def plot_horizon_line(im, pt1, pt2):
    """
    Plots the horizon line.
    """
    # <YOUR IMPLEMENTATION>
    plt.figure()
    plt.imshow(im)
    pt1 = pt1 / pt1[2]
    pt2 = pt2 / pt2[2]
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')
    plt.show()
    

In [20]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR IMPLEMENTATION>

    v1 = vpts[:, 0][:, np.newaxis]
    v2 = vpts[:, 1][:, np.newaxis]
    v3 = vpts[:, 2][:, np.newaxis]


    f, px, py = symbols('f, px, py')
    # k = Matrix([[f, 0, px], [0, f, py], [0, 0, 1]])
    k = Matrix([[1 / f, 0, -px / f], [0, 1 / f, -py / f], [0, 0, 1]])


    # ik = inv(k)

    e1 = v1.T * k.T * k * v2
    e2 = v1.T * k.T * k * v3
    e3 = v2.T * k.T * k * v3




    f, px, py = solve([e1[0], e2[0], e3[0]], (f, px, py))[0]

    return abs(f), px, py

In [21]:
def get_rotation_matrix(f, u, v, vpts):
    """
    Computes the rotation matrix using the camera parameters.
    """
    v1 = vpts[:, 0][:, np.newaxis]
    v2 = vpts[:, 1][:, np.newaxis]
    v3 = vpts[:, 2][:, np.newaxis]
    
    k = np.array([[1 / f, 0, u / f], [0, 1 / f, v / f], [0, 0, 1]]).astype(np.float)


    r1 = np.dot(k, v1)
    r2 = np.dot(k, v2)
    r3 = np.dot(k, v3)
    
    
    return np.concatenate((r1 / norm(r1), r2 / norm(r2), r3 / norm(r3)), axis=1)

In [22]:
def estimate_height(person, object, horizon, im, v3, p_height, v1, v2):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # <YOUR IMPLEMENTATION>
   

    p_u = person[:, 0]
    p_d = person[:, 1]

    o_u = object[:, 0]
    o_d = object[:, 1]
    o_line = np.cross(o_u, o_d)

    d_line = np.cross(p_d, o_d)
    d_i = np.cross(d_line, horizon)
    d_i = d_i /d_i[-1]
    t_line = np.cross(d_i, p_u)
    intersect = np.cross(o_line, t_line)
    intersect = intersect/ intersect[-1]

    height  = p_height * norm(o_u-o_d) * norm(v3 - intersect) / norm(intersect - o_d) / norm(v3 - o_u)
    
    plt.figure()
    plt.imshow(im)
    pt1 = v1 / v1[2]
    pt2 = v2 / v2[2]
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')
    pt1 = p_d
    pt2 = d_i
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')
    pt1 = p_u
    pt2 = d_i
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

    pt1 = o_u
    pt2 = d_i
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

    pt1 = o_d
    pt2 = d_i
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')
    plt.show()

    print(height)


    return height

# Main function

In [23]:
im = np.asarray(Image.open('abc.jpeg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines)
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])

print("vpts")
print(vpts)

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(im, vpts[:, 0], vpts[:, 1])

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)

print(f, u, v)

# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(f, u, v, vpts)

print(R)

# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(coords['person'], coords[obj], horizon_line, im, vpts[:, 2],  5.6  * 0.304, vpts[:, 0], vpts[:, 1])
    height = estimate_height(coords['person'], coords[obj], horizon_line, im, vpts[:, 2],  6  * 0.304, vpts[:, 0], vpts[:, 1])


Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
547.1907671017241 334.8100395996627
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
-96.98234354038404 310.8056340053248
Getting vanishing point 2
Set at least 3 lines to compute vanishing

/var/folders/2d/mqx8_d3j7sz1dwfnhtqnd3c00000gn/T/ipykernel_72351/2627096133.py:24: RuntimeWarning: divide by zero encountered in divide
  pt1 = pt1 / pt1[2]
/var/folders/2d/mqx8_d3j7sz1dwfnhtqnd3c00000gn/T/ipykernel_72351/2627096133.py:24: RuntimeWarning: invalid value encountered in divide
  pt1 = pt1 / pt1[2]
/var/folders/2d/mqx8_d3j7sz1dwfnhtqnd3c00000gn/T/ipykernel_72351/2627096133.py:25: RuntimeWarning: divide by zero encountered in divide
  pt2 = pt2 / pt2[2]
/var/folders/2d/mqx8_d3j7sz1dwfnhtqnd3c00000gn/T/ipykernel_72351/2627096133.py:25: RuntimeWarning: invalid value encountered in divide
  pt2 = pt2 / pt2[2]


286.128897787978 323.180489140817 211.474554894614
[[-0.81594976  0.35508034 -0.85582031]
 [-0.51212719  0.81986272  0.36761151]
 [-0.26823819  0.44915818 -0.36391397]]
Click on the top coordinate of person


/var/folders/2d/mqx8_d3j7sz1dwfnhtqnd3c00000gn/T/ipykernel_72351/1975237851.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  k = np.array([[1 / f, 0, u / f], [0, 1 / f, v / f], [0, 0, 1]]).astype(np.float)


Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts
Estimating height of CSL building
34.32900955418927
36.781081665202784
Estimating height of the spike statue
16.08847771665782
17.237654696419092
Estimating height of the lamp posts
4.165466738116066
4.4630000765529285
